In [ ]:
%run setup

![MLFlow](https://images.squarespace-cdn.com/content/v1/561c001de4b06530bb65f080/1591324514062-057XFNW1NEPYG0V2W0FK/ke17ZwdGBToddI8pDm48kK6mKuWQ1p4ESdjwtW3BA1EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKcFkXF-TzreDwzLgtTwibDCbWtdkc1DXP09IipfwxkgNMvACCePm4cJPaApzbw6Bn2/MLflow-logo-final-white-TM.jpg)

# Modellsporing med MLFlow

To komponenter/deler som må være på plass for et fungerende oppsett:
1. *En "tracking server"* - en applikasjon kjørende "sentralt" som kan motta resultater fra python-programmer i prosjektet som logger MLFlow-resultater
    - Tracking serveren kan også kjøres lokalt, ved behov, på egen maskin
2. *En pakke i prosjektet* - og bruk av denne for å logge resultater til tracking serveren

Merk: 
   - MLFlow har flere komponenter enn dette som kan hjelpe med andre deler av arbeidsflyten, men modellsporing er nok den mest brukte (og subjektivt sett nyttige) delen av MLFlow
   - Å legge til modellsporing i MLFlow er dessuten ikke spesielt "inngripende", og man kan ofte (i stor grad) beholde samme struktur/arkitektur som tidligere

# Først, litt gjenkjennelig ML-kode

## Innhenting av data

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset="all")
len(newsgroups.data)

## Oppretting av en enkel NLP-pipeline

Vi lager her en enkel NLP-pipeline som vektoriserer tekstene med TF-IDF, og deretter mater dette til en SVM.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

analyzer = "word"
stop_words = "english"
ngram_range = (1, 2)
min_df = 3
max_df = .8
max_features = 10_000
alpha = 0.0001
max_iter = 1000
loss = "hinge"
penalty = "l2"

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        analyzer=analyzer,
        stop_words=stop_words,
        ngram_range=ngram_range,
        min_df=min_df,
        max_df=max_df,
        max_features=max_features
    )),
    ("clf", SGDClassifier(
        alpha=alpha,
        penalty=penalty,
        loss=loss,
        max_iter=max_iter,
        n_jobs=-1
    ))
])

# Et "MLFlow run"

Nå som vi har en utrent pipeline, og data på plass er vi klare for å *kjøre*. Da er det på tide med første kall/integrasjon mot MLFlow. Vi må fortelle MLFlow at vi ønsker et "run".

Et "run" er som navnet sier en *kjøring* - det vil si kjøring av trening og/eller evaluering av en modell. Resultatet av en kjøring vil være et sett med *filer* ("artefakter") og *metrikker* som man ønsker å ta vare på og kunne sammenligne på tvers av kjøringer. Knyttet til en kjøring er også *parametere* som beskriver konfigurasjon som påvirker ytelsen til modellen og *tager* som beskriver annen metadata ved modellen.

In [ ]:
import mlflow
mlflow.start_run()

## Logging av parametere

Vi kan starte med å logge det vi allerede vet om kjøringen - hvilke parametre som har blitt brukt for å konfigurere modellen. For å logge dette kan vi bruke `mlflow.log_param` (eller `.log_params` for å logge flere på en gang vha en dict):

In [ ]:
mlflow.log_param("tfidf__analyzer", analyzer)
mlflow.log_param("tfidf__stop_words", stop_words)
mlflow.log_param("tfdif__ngram_range", ngram_range)
mlflow.log_param("tfidf__min_df", min_df)
mlflow.log_param("tfdif__max_df", max_df)
mlflow.log_param("tfidf__max_features", max_features)
mlflow.log_param("clf__alpha", alpha)
mlflow.log_param("clf__max_iter", max_iter)
mlflow.log_param("clf__loss", loss)
mlflow.log_param("clf__penalty", penalty)

## Tagging

*Parametere* er konfigurasjon som påvirker modellen, men vi ønsker ofte å knytte annen type metadata til en kjøring. Dette kan f.eks. være metadata som *docker*-taggen til bildet som ble brukt for kjøringen. For å sette tagger kan vi bruke `mlflow.set_tag` (eller `.set_tags` for å logge flere på en gang vha en dict):

In [ ]:
mlflow.set_tag("docker_image_tag", "a1b2c3d4")

## Logging av metrikker

Vi starter så med å kjøre kryss-validering, for evaluering av modellen:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from functools import partial
from mlflow_demo.metrics import Scorer
import numpy as np

scorer = Scorer(
    precision_weighted=partial(f1_score, average="weighted", zero_division=0),
    recall_weighted=partial(f1_score, average="weighted", zero_division=0),
    f1_weighted=partial(f1_score, average="weighted", zero_division=0)
)

cv = StratifiedKFold(n_splits=3, random_state=419, shuffle=True)

target_names = np.asarray(newsgroups.target_names)
X, y = newsgroups.data, target_names[newsgroups.target]

metrics_per_split = cross_validate(pipeline, X, y, scoring=scorer, cv=cv, verbose=10)

Vi har nå følgende metrikker:

In [ ]:
metrics_per_split

Disse kan vi så logge til mlflow med `mlflow.log_metric` (eller `.log_metrics` for å logge flere på en gang vha en dict):

In [ ]:
from statistics import mean

for metric_name, metric_values in metrics_per_split.items():
    metric_name = metric_name.replace("test_", "")
    mlflow.log_metric(metric_name, mean(metric_values))

## Logging av filer/"artefakter"

Under kjøring skaper man ofte filer som det kan være interessant å ta vare på til senere. Dette kan for eksempel være figurer, rapporter, eller filer som inneholder selve prediksjonene gjort av modellen. Alle filer kan logges til et "run". Noen typer, slik som figurer og tekst kan logges med egne funksjoner, resterende filtyper kan logges med den generiske funksjonen `mlflow.log_artifact`.

### Logging av en figur

In [ ]:
from mlflow_demo.plotting import create_confusion_matrix_fig
import plotly.offline as pyo

pyo.init_notebook_mode()

confusion_matrix_fig = create_confusion_matrix_fig(scorer.y_, scorer.y_pred_)
confusion_matrix_fig

In [ ]:
mlflow.log_figure(confusion_matrix_fig, "confusion_matrix.html")

### Logging av tekstfiler

In [ ]:
from sklearn.metrics import classification_report

clf_report = classification_report(scorer.y_, scorer.y_pred_)
print(clf_report)

In [ ]:
mlflow.log_text(clf_report, "classification_report.txt")

### Logging av modellen

Kanskje det viktigste å ta vare på til senere er selve modellen - i hvert fall dersom man bruker MLFlow i et produksjonssystem. MLFlow har støtte for å logge modell-objekter fra stort sett alle større ML-rammeverk. For eventuelle rammeverk som ikke søttes har man dessuten mulughet til å logge modellen som en generisk "python-funksjon". Man kan selvsagt også skrive filene på valgfri måte selv, og logge dem via `mlflow.log_artifact`.

I denne notebooken har vi brukt scikit-learn, og benytter dermed funksjoner for scikit-learn for logging av modellen.

In [ ]:
pipeline.fit(X, y)
pipeline.predict(["Computers will rule the world"])

In [ ]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

signature = ModelSignature(
    inputs=Schema([ColSpec("string", "text")]),
    outputs=Schema([ColSpec("string", "category_name")])
)
mlflow.sklearn.log_model(pipeline, "model", signature=signature)

### Avslutte en kjøring

In [ ]:
mlflow.end_run()

# Hyperparameter-tuning av modellen

En av de virkelige styrkene til MLFlow er å kunne bruke det for å sammenligne kjøringer med ulike parametere. La oss lage en funksjon for å gjennomføre en kjøring gitt et sett av parametere, som logger paramteterne og resultater fra kjøringen:

In [ ]:
from sklearn.base import clone

def evaluate_and_log(params):
    with mlflow.start_run():
        mlflow.log_params(params)
        
        _pipeline = clone(pipeline).set_params(**params)
        _scorer = scorer.copy()
        
        metrics_per_split = cross_validate(_pipeline, X, y, scoring=_scorer, cv=cv)
        
        for metric_name, metric_values in metrics_per_split.items():
            metric_name = metric_name.replace("test_", "")
            mlflow.log_metric(metric_name, mean(metric_values))
            
        confusion_matrix_fig = create_confusion_matrix_fig(_scorer.y_, _scorer.y_pred_)
        mlflow.log_figure(confusion_matrix_fig, "confusion_matrix.html")
        
        clf_report = classification_report(_scorer.y_, _scorer.y_pred_)
        mlflow.log_text(clf_report, "classification_report.txt")

Så definerer vi et rom vi ønsker å søke over:

In [ ]:
from sklearn.model_selection import ParameterGrid
import random

params_grid = {
    "tfidf__analyzer": ["word", "char", "char_wb"],
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3), (3, 3), (4, 4), (5, 5)],
    "tfidf__min_df": [1, 2, 5, 10],
    "tfidf__max_df": [0.5, 0.8, 1.0],
    "tfidf__stop_words": ["english", None],
    "tfidf__max_features": [5000, 10_000, 20_000, None],
    "clf__alpha": [0.0001, 0.001, 0.01, 0.1],
    "clf__max_iter": [100, 500, 1000, 2000, 5000],
    "clf__loss": ["log", "hinge"],
    "clf__penalty": ["l1", "l2", "elasticnet"]
}


def is_valid_param_combo(params):
    analyzer = params["tfidf__analyzer"]
    ngram_range_min, _ = params["tfidf__ngram_range"]
    if ngram_range_min < 3:
        return analyzer == "word"
    return analyzer.startswith("char")

params_list = list(filter(is_valid_param_combo, ParameterGrid(params_grid)))
params_samples = random.sample(params_list, 200)
params_samples[0]

## Kjøring av tuning

Vi definerer først et nytt annet MLFLow-eksperiment hvor vi ønsker at disse kjøringene skal ende opp. Man kan tenke på et MLFLow-eksperiment som en mappe/samling av kjøringer som man ønsker å kunne sammenligne. Typisk bør dermed alle kjøringene innenfor samme eksperiment være *sammenlignbare* - altså logge samme parametere og metrikker.

In [ ]:
mlflow.set_experiment("Simple NLP model")

In [ ]:
from tqdm import tqdm

for params in tqdm(params_samples):
    evaluate_and_log(params)